In [1]:
#from keras.models import load_model
import json
from LJT_database.feature_prep import feature_preperation
from LJT_database.merge_dataset import retrieve_data_predictors
from LJT_helper_functions.dataset_prep import prepare_dataset_prediction
import warnings
from deep_learning_models.training_models import build_model
from deep_learning_models.feature_selection import make_components_pca
import sys
from LJT_database.firestore_codes import  add_update,retrieve_updates
import platform
from LJT_helper_functions.helpers import send_message_telegram, convert_datatype
from LJT_database.merge_dataset import retrieve_data_predictors
from LJT_database.update_table import insert_rows
from LJT_database.feature_prep import feature_preperation
import pandas as pd
warnings.filterwarnings('ignore')

In [3]:
dataset_prepared = pd.read_csv("data_lex.csv")

In [4]:
dataset_prepared = dataset_prepared[[col for col in dataset_prepared.columns if col != 'close_price_next_min']]

In [5]:
general_settings={
                    'coin': 'ETHBTC',
                    'bins': True,
                    'training_size': 0.8,
                    'algorithm': 'LSTM',
                    'include_transaction_cost': False,
                    'minutes_lookback':0
                 }

In [6]:
general_settings['columns'] = dataset_prepared.columns
general_settings['experiment_date'] ="lex_testertje_38" #let op met deze aanpassen. Graag niets wat dag_maand zegt tussen 20_05 / 15_06
len(dataset_prepared.columns)

344

In [7]:
#only run once or when updating stuff in dataset etc. Pca, normalizer, etc are stored in Storage
"""
from deep_learning_models.feature_selection import make_components_pca
make_components_pca(dataset_prepared,
                    general_settings,
                    percentage_variance=.99,
                    type_scaler='min_max_scaler',
                    min_cor=0.4) #deze is handmatig uitgezet no stress:) 
"""

"\nfrom deep_learning_models.feature_selection import make_components_pca\nmake_components_pca(dataset_prepared,\n                    general_settings,\n                    percentage_variance=.99,\n                    type_scaler='min_max_scaler',\n                    min_cor=0.4) #deze is handmatig uitgezet no stress:) \n"

In [8]:
from deep_learning_models.feature_selection import create_reduced_features
df = create_reduced_features(dataset_prepared,general_settings)

Blob ethbtc/lex_testertje_38_scaler.p downloaded to temp_store/ethbtc/lex_testertje_38_scaler.p.
Blob ethbtc/lex_testertje_38_pca.p downloaded to temp_store/ethbtc/lex_testertje_38_pca.p.
Blob ethbtc/lex_testertje_38_columns_scaler.p downloaded to temp_store/ethbtc/lex_testertje_38_columns_scaler.p.
Blob ethbtc/lex_testertje_38_columns_in_pca.p downloaded to temp_store/ethbtc/lex_testertje_38_columns_in_pca.p.


In [9]:
settings_analysis = {
    "algoirthm":"LSTM",
    "experiment_date":"11_6",
    "neurons":75,
    "dropout":0.37644318657738224,
    "loss_func": "binary_crossentropy",
    "activation_function":"softmax",
    "number_layers":1,
    "batch_size":2520,
    "epochs":500,
    "bidrectional":False,
    "window_size":113,
    "time_ahead_prediction":1,
    "optimizer":'adam',
    "learning_rate": 0.000526661,
}

In [10]:
model = build_model(settings_analysis,general_settings,df,save_model=True,normalized=True)
model.save("model_lex.h5")

Using TensorFlow backend.


---------------------------------NEW EXPERIMENT -----------------------------------------------
current time: 2:0
--data--
length training data =  55773
percentage True train = 0.4805909669553368
length test data =  13944
percentage True test = 0.4776247848537005
highest_val_ac 0.5231640934944153
{'sensitivity': 0.0867867867867868, 'specificity': 0.9221581548599671, 'precision': 0.5048034934497817, 'accuracy': 0.5231640849110729, 'f_1': 0.14811018577834723}
Running time  0:12:59.276020
